In [2]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 885.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 11.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 10.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5

In [1]:
import os
import nibabel as nib
import numpy as np
import tensorflow

2023-12-21 09:51:08.385620: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras import models, layers

In [1]:
# Set your data directories
train_images_dir = '../aocr2024/1_Train,Valid_Image/'
train_masks_dir = '../aocr2024/1_Train,Valid_Mask/'

batch_size = 16
n_epoch = 3
val_steps = 3

# Define a function to load and preprocess NIfTI images
def load_nifti_image(file_path):
    image = nib.load(file_path).get_fdata()
    # Add any necessary preprocessing steps here
    return image

# Create data generators for training and validation
def generate_data_generator(images_dir, masks_dir, subset='training', batch_size=batch_size):
    #image_files = [os.path.join(images_dir, file) for file in os.listdir(images_dir)]
    #mask_files = [os.path.join(masks_dir, file) for file in os.listdir(masks_dir)]

    # Assume the images and masks have the same file names
    file_names = [os.path.splitext(file)[0] for file in os.listdir(images_dir)]
    file_names.sort()

    # Split into training and validation sets
    split_index = int(len(file_names) * 0.8)
    if subset == 'training':
        file_names = file_names[:split_index]
    else:
        file_names = file_names[split_index:]

    while True:
        for i in range(0, len(file_names), batch_size):
            batch_files = file_names[i:i + batch_size]
            batch_images = [load_nifti_image(os.path.join(images_dir, f + '.gz')) for f in batch_files]
            batch_masks = [load_nifti_image(os.path.join(masks_dir, f + '.gz')) for f in batch_files]

            yield (np.array(batch_images), np.array(batch_masks))

# Define the simplified 2D U-Net model
def simple_2d_unet_model(input_shape=(512, 512, 90)):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Mid-level
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)

    # Decoder
    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    concat1 = layers.concatenate([up1, conv2], axis=-1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(concat1)

    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    concat2 = layers.concatenate([up2, conv1], axis=-1)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(concat2)

    # Output layer
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv5)

    model = models.Model(inputs=inputs, outputs=outputs)

    return model

# Instantiate the 2D U-Net model
simple_2d_model = simple_2d_unet_model()

# Compile the model
simple_2d_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Set the number of steps per epoch based on your batch size and the number of training samples
num_train_samples = len(os.listdir(train_images_dir))
steps_per_epoch = num_train_samples // batch_size

# Create data generators
train_data_generator = generate_data_generator(train_images_dir, train_masks_dir, subset='training', batch_size=batch_size)
validation_data_generator = generate_data_generator(train_images_dir, train_masks_dir, subset='validation', batch_size=batch_size)

# Train the model
simple_2d_model.fit(
    train_data_generator, 
    steps_per_epoch=steps_per_epoch, 
    epochs=n_epoch, 
    validation_data=validation_data_generator, 
    validation_steps=val_steps
)


NameError: name 'layers' is not defined

Steps to try: 
- Reduce to 1 epoch
- Fit on test data
- Get results formatted as expected 
- Submit

Then go back and adjust model!